# ASTR596, FDS: Homework set 3 - Moving from MLE and Frequentism to evaluating the Posterior distribution and Bayesian Statistics



## Problem 1

Last week you showed that Bayesian and Frequentist approaches are often equivalent for simple problems (after all you implicitly had a prior in the form of a grid). 

But it is also true that they can diverge greatly. In practice, this divergence makes itself most clear in two different situations:

1. The handling of "nuisance parameters"
2. The subtle (and often overlooked) difference between frequentist confidence intervals and Bayesian credible intervals

Here, we focus on the first point: the difference between frequentist and Bayesian treatment of nuisance parameters.


## What is a Nuisance Parameter?

***A nuisance parameter is any quantity whose value is not relevant to the goal of an analysis, but is nevertheless required to determine some quantity of interest***.

For example, last week, in problem 3, we estimated both the mean $C$ and shape $\sigma_\text{source}$ for some spherical cow galaxy. Now the shape might be important, but in homework 1, you only needed the brightness of the sources where you plotted up $r$ vs $g-i$ for galaxies (you needed the shape to separate stars from galaxies in the form of MEAN_OBJECT_TYPE, but that's it).

As far as you were concerned in homework 1, the shape was a **nuisance** parameter.


## A Classic Problem:


We'll start with an example of nuisance parameters that, in one form or another, dates all the way back to the posthumous [1763 paper](http://www.stat.ucla.edu/history/essay.pdf) written by Thomas Bayes himself. The particular version of this problem we'll study is borrowed from [Eddy 2004](ftp://selab.janelia.org/pub/publications/Eddy-ATG3/Eddy-ATG3-reprint.pdf).

The setting is a rather contrived game in which Alice and Bob bet on the outcome of a process they can't directly observe:

> Alice and Bob enter a room. Behind a curtain there is a billiard table, which they cannot see, but their friend Carol can. Carol rolls a ball down the table, and marks where it lands. Once this mark is in place, Carol begins rolling new balls down the table. If the ball lands to the left of the mark, Alice gets a point; if it lands to the right of the mark, Bob gets a point.  We can assume for the sake of example that Carol's rolls are unbiased: that is, the balls have an equal chance of ending up anywhere on the table.  **The first person to reach six points wins the game.**

Here ***the location of the mark (determined by the first roll) can be considered a nuisance parameter***.

It is unknown, and not of immediate interest, but it clearly must be accounted for when predicting the outcome of subsequent rolls. If the first roll settles far to the right, then subsequent rolls will favor Alice. If it settles far to the left, Bob will be favored instead.

Given this setup, here is the question we ask of ourselves:

> In a particular game, after eight rolls, Alice has five points and Bob has three points. What is the probability that Bob will go on to win the game?

Intuitively, you probably realize that because Alice received five of the eight points, the marker placement likely favors her. And given this, it's more likely that the next roll will go her way as well. 

And she has three opportunities to get a favorable roll before Bob can win; she seems to have clinched it.  But, **quantitatively**, what is the probability that Bob will squeak-out a win?


Someone following a classical frequentist approach might reason as follows:

To determine the result, we need an intermediate estimate of where the marker sits. We'll quantify this marker placement as a probability $p$ that any given roll lands in Alice's favor.  Because five balls out of eight fell on Alice's side of the marker, we can quickly show that the maximum likelihood estimate of $p$ is given by:

$$
\hat{p} = 5/8
$$

(This result follows in a straightforward manner from the [binomial likelihood](http://en.wikipedia.org/wiki/Binomial_distribution)). 

# 1.1 Under the assumptions of maximum likelihood, what is the probability of Bob winning the game - i.e. getting six points, and what are the odds 
# (i.e. $(1 - p_\text{Bob wins})/p_\text{Bob wins}$)

Bob needs to pray to whatever god he believes in and get 3 rolls in a row in his favor. This is represented by
> $p_{Bob wins} = (1-p_{Alice wins})^3 = (1-\hat{p})^3$

where the cube comes from having 3 consecutive rolls.

In [19]:
prob_alice_win = 5/8 #this is p_hat
prob_bob_win = (1 - prob_alice_win)**3 
odds_agst_bob = (1 - prob_bob_win) / prob_bob_win

print("Bob's chance of winning is...", prob_bob_win)
print("The odds against Bob winning is... {0:.0f} to 1".format(odds_agst_bob))

Bob's chance of winning is... 0.052734375
The odds against Bob winning is... 18 to 1


We can also approach this problem from a Bayesian standpoint. This is slightly more involved, and requires us to first define some notation.

We'll consider the following random variables:

- $B$ = Bob Wins
- $D$ = observed data, i.e. $D = (n_A, n_B) = (5, 3)$
- $p$ = unknown probability that a ball lands on Alice's side during the current game

We want to compute $P(B~|~D)$; that is, the probability that Bob wins given our observation that Alice currently has five points to Bob's three.

The general Bayesian method of treating nuisance parameters is ***marginalization***, or integrating the joint probability over the entire range of the nuisance parameter. In this case, that means that we will first calculate the joint distribution
$$
P(B,p~|~D)
$$
and then marginalize over $p$ using the following identity:
$$
P(B~|~D) \equiv \int_{-\infty}^\infty P(B,p~|~D) {\mathrm d}p
$$
This identity follows from the definition of conditional probability, and the law of total probability: that is, it is a fundamental consequence of probability axioms and will always be true. Even a frequentist would recognize this; they would simply disagree with our interpretation of $P(p)$ as being a measure of uncertainty of our own knowledge.

To compute this result, we will manipulate the above expression for $P(B~|~D)$ until we can express it in terms of other quantities that we can compute.

We'll start by applying the following definition of [conditional probability](http://en.wikipedia.org/wiki/Conditional_probability#Definition) to expand the term $P(B,p~|~D)$:

$$
P(B~|~D) = \int P(B~|~p, D) P(p~|~D) dp
$$

Next we use [Bayes' rule](http://en.wikipedia.org/wiki/Bayes%27_theorem) to rewrite $P(p~|~D)$:

$$
P(B~|~D) = \int P(B~|~p, D) \frac{P(D~|~p)P(p)}{P(D)} dp
$$

Finally, using the same probability identity we started with, we can expand $P(D)$ in the denominator to find:

$$
P(B~|~D) = \frac{\int P(B~|~p,D) P(D~|~p) P(p) dp}{\int P(D~|~p)P(p) dp}
$$

Now the desired probability is expressed in terms of three quantities that we can compute. Let's look at each of these in turn:
<small>
- $P(B~|~p,D)$: This term is exactly the frequentist likelihood we used above. In words: given a marker placement $p$ and the fact that Alice has won 5 times and Bob 3 times, what is the probability that Bob will go on to six wins?  
    
    
- $P(D~|~p)$: this is another easy-to-compute term. In words: given a probability $p$, what is the likelihood of exactly 5 positive outcomes out of eight trials? The answer comes from the well-known [Binomial distribution](http://en.wikipedia.org/wiki/Binomial_distribution)
    
    
- $P(p)$: this is our prior on the probability $p$. By the problem definition, we can assume that $p$ is evenly drawn between 0 and 1.  That is, $P(p) \propto 1$, and the integrals range from 0 to 1.</small>

# 1.2 Evaluate that integral and get an expression in terms of $p$

# paleo's answer:

Currently we have our desired probability as
$$
P(B~|~D) = \frac{\int P(B~|~p,D) P(D~|~p) P(p) dp}{\int P(D~|~p)P(p) dp}.
$$

Thankfully, we already calculated $P(B~|~p,D)$ in part 1.1. This is simply $P(B~|~p,D)=(1-p)^3$. :)

Also, we know the prior to be $P(p) \propto 1$ (for p evaluated between 0 and 1), and so this makes things easy as it does not affect the integrand. It is not dependent on $p$.

Lastly, we need to find $P(D~|~p)$. Well, this is simply from the Binomial distribution's pmf. The probability of getting exactly $k$ successes in $n$ independent Bernoulli trials is given by the pmf is for our situation
$$
{n \choose k} p^k (1-p)^{(n-k)} = {8 \choose 5} p^5 (1-p)^{(8-5)} ={8 \choose 5} p^5 (1-p)^{3}.
$$
Because the expression $P(D~|~p)$ is in the numerator and denominator and doesn't depend on $p$, the ${8 \choose 5}$ part cancels out. Thus, our expression is found via substitution:
$$
P(B~|~D) = \frac{\int P(B~|~p,D) P(D~|~p) P(p) dp}{\int P(D~|~p)P(p) dp} = \frac{\int_0^1 p^5 (1-p)^6 dp}{\int_0^1 p^5 (1-p)^3 dp}
$$

The integrals you find might look a bit difficult, but they are just special cases of the [Beta Function](http://en.wikipedia.org/wiki/Beta_function):
$$
\beta(n, m) = \int_0^1 (1 - p)^{n - 1} p^{m - 1}
$$

scipy has an implementation of this in `scipy.special`.

# 1.3 Evaluate your expression numerically and compute the probability and the odds that Bob wins

In [20]:
# paleo's answer here

from scipy.special import beta
bayes_prob_bob_win = (beta(6, 7) / beta(6, 4)) # +1 to balance out -1 in beta function for scipy
odds_against_B = (1 - bayes_prob_bob_win) / bayes_prob_bob_win
                                                      

print("The probability of Bob winning is given by... P(B|D) = {0:.3f}".format(bayes_prob_bob_win))
print("The odds against Bob winning is... {0:.0f} to 1".format(odds_against_B))

The probability of Bob winning is given by... P(B|D) = 0.091
The odds against Bob winning is... 10 to 1


Wait, this is better than 18 to 1!

# 1.4 Finally, lets check the result using an approach called Monte Carlo, where we simulate a bunch of games, and simply count the fraction of relevant games that Bob goes on to win. The current problem is especially simple because so many of the random variables involved are uniformly distributed. 

* Simulate 100,000 random p between 0 and 1 - this will be a 1D array
* given each p, to win the game needs *at most* 11 rolls - simulate 11 rolls for each p - this will be a 2D array
* count the cumultative wins for Alice and Bob at each roll - this is a 2D array
* determine which games has Bob with three points by the end of game 8 - this is a 1D array
* considering only these games, find the number of games which Bob won (i.e. has six points at the end of game 11)
* compute the total probability that Bob won, and the odds that Bob won.


## You don't need anything more than `numpy` to do this - in particular `numpy.random` and `numpy.cumsum` to do this.
No for loops.

In [17]:
# paleo's answer

import numpy as np
np.random.seed(10) #random seed is 10 b/c it's my favorite number
p = np.random.uniform(0,1,100000) #100k games

#winner needs at most 11 rolls for each p - 2D array
roll_tot = np.random.random((11, 100000))
#cumulative wins for Bob, Alice - 2D array
bob_win_N = np.cumsum(roll_tot >= p, 0) #if roll count hits 11, Bob wins (cause he gets next 3 from 8)
alice_win_N = np.cumsum(roll_tot < p, 0) #if roll count < 11, Alice wins

In [16]:
# get number of games to get us in this situation
playable = bob_win_N[7] == 3 #Bob gets next 3 rolls after the 8th (remember python index starts @ 0)
playable_tot = np.sum(playable) #total playable games out of 100k

# out of playable games, how many did bob win? alice win?
bob_win_N = bob_win_N[:, playable]
#print(bob_win_N)
alice_win_N = alice_win_N[:, playable]
bob_won = np.sum(np.sum(bob_win_N[10] == 6)) # bob gets 6 points to win out of 11 rolls
print("# games Bob won is... {0}".format(bob_won), "out of", playable_tot, "playable games from 100000")

prob_bob_win = bob_won * (1 / np.sum(playable)) # compute total probability bob win
odds_against_bob = (1 - prob_bob_win) / prob_bob_win

print("prob bob win(!) is... {0:.3f}".format(prob_bob_win))
print("The odds against Bob winning is... {0:.0f} to 1".format(odds_against_bob))

# games Bob won is... 972 out of 11073 playable games from 100000
prob bob win(!) is... 0.088
The odds against Bob winning is... 10 to 1


## 1.5 Why do the two results disagree?

# paleo's answer

In short, the Bayesian result agreed with the brute force Monte Carlo result at 10 to 1 odds against Bob winning, but differed from the Frequentist approach at 18 to 1 odds. This seems to be because of the nuisance parameter, and how the frequentist approach deals with it (or rather, **doesn't**) in this problem. Earlier, we defined $p$ as the "unknown probability that a ball lands on Alice's side during the current game", and the nuisance parameter to be "the location of the mark (determined by the first roll)". In this setup, $p$ and the nuisance parameter are linked in that the value of $p$ depends very much on where the location of the initial ball is, and in the frequentist approach there's no way to take this info into account when calculating our probability- we just say "hey, this is an independent event like rolling die, where previous rolls don't depend on anything before", whereas in the Bayesian approach we do take this into account and marginalize over $p$.

Basically, there is further information with which we can refine our probability estimate but we don't do so in naive frequentist approach-- we ignore the nuisance parameter in general and assume $p$'s independence. However, in reality there is some tie in between the two and we need to do some algebraic wizardry to account for it, which is what the Bayesian (and brute MC) approach does. Maybe there's a way to mitigate the problem's dependence on $p$ to get a more similar answer between the different methods?